In [11]:
{-# LANGUAGE RecordWildCards #-}
import GHC.Generics
import Torch
import Torch.NN as NN
import Torch.Functional as F
import Torch.Functional.Internal as FI

import Torch.TensorFactories
import Torch.TensorOptions

import Control.Monad (when)

In [12]:
data Config = Config
  { configNEmbd :: Int
  } deriving (Show, Eq)

Line 1: Use newtype instead of data
Found:
data Config
  = Config {configNEmbd :: Int}
  deriving (Show, Eq)
Why not:
newtype Config
  = Config {configNEmbd :: Int}
  deriving (Show, Eq)

In [13]:
data MLP = MLP
  { fcLayer :: Linear
  , projLayer :: Linear
  , nEmbd :: Int
  } deriving (Generic, Show)

In [14]:
mlpInit :: Config -> IO MLP
mlpInit Config{..} = do
  
  fcLayer <- sample (LinearSpec configNEmbd (4 * configNEmbd))
  projLayer <- sample (LinearSpec (4 * configNEmbd) configNEmbd)
     
  return MLP
    { fcLayer = fcLayer
    , projLayer = projLayer
    , nEmbd = configNEmbd
    }

In [ ]:
mlpForward :: MLP -> Tensor -> Tensor
mlpForward MLP{..} x = let
 fc = FI.gelu $ NN.linear fcLayer x
 proj = NN.linear projLayer fc
  in 
   proj

In [22]:
batchSize = 64
embdSize = 128

In [23]:
mlp <- mlpInit (Config embdSize)

In [24]:
x_example <- randnIO' [batchSize, embdSize]
putStrLn $ "x_example:" ++ show (shape x_example)

x_example:[64,128]

In [26]:
output = mlpForward mlp x_example
putStrLn $ "output:" ++ show (shape output)


output:[64,128]